# Calibration Rectangle 

150mm x 50mm rectangle for calibration purposes
splay the image.

In [7]:
import gcode
import numpy as np
import matplotlib.pyplot as plt

In [8]:
import requests

esphome_cfg = {
    "host": "cnc.lan",
    "id": "cnc_relay"
}

base_url = f"http://{esphome_cfg['host']}/switch/{esphome_cfg['id']}"
base_url

'http://cnc.lan/switch/cnc_relay'

In [9]:
r = requests.post(f"{base_url}/turn_on")
assert r.status_code == 200

r = requests.get(f"{base_url}")
assert r.json()["state"] == "ON"
assert r.json()["value"]



ConnectionError: HTTPConnectionPool(host='cnc.lan', port=80): Max retries exceeded with url: /switch/cnc_relay/turn_on (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f07384e2eb0>: Failed to establish a new connection: [Errno 113] No route to host'))

In [4]:
import sys

# https://github.com/jed-frey/build_opencv
sys.path.append("/opt/opencv4/lib/python3.8/site-packages")

import cv2
from PIL import Image
from IPython.display import display

In [5]:
import timeout_decorator

class VideoCapture2(cv2.VideoCapture):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    @property
    def frame(self):
        ret, frame = self.read()
        if ret:
            return frame
        else:
            raise Exception("Reading Failed")
        
    def pil_img(self):
        img = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
        return Image.fromarray(img)
    
    def __enter__(self):
        return self
    
    @timeout_decorator.timeout(10)
    def __exit__(self,type, value, traceback):
        while self.isOpened():
            self.release() 

            
import IPython.core.magic
@IPython.core.magic.register_line_magic
def webcam(*args, **kwargs):
    webcams = [arg for arg in args[0].split(" ")]
    for webcam in webcams:
        if webcam.startswith("/dev/video"):
            webcam = int(webcam.split("/dev/video")[-1])
        else:
            try:
                webcam = int(webcam)
            except:
                pass
        with VideoCapture2(webcam) as cap:
            img = cap.pil_img()
            display(img)

In [6]:
import os
auth = os.environ["WEBCAM_AUTH"]
ip   = os.environ["WEBCAM_IP"]

In [ ]:
%webcam rtsp://$auth@$ip/cam/realmonitor?channel=1&subtype=0

In [7]:
r = requests.post(f"{base_url}/turn_on")
assert r.status_code == 200

r = requests.get(f"{base_url}")
assert r.json()["state"] == "ON"
assert r.json()["value"]

import grbl

with grbl.Grbl(port="/dev/ttyUSB0") as cnc:
    cnc.home()

In [5]:
def init():
    prog = gcode.GCode()
    prog.G92(X=0, Y=0, Z=0) 
    prog.G21() # Metric
    prog.G90() # Absolute
    return prog


X_pos = 10
laserin_speed=200
doe=gcode.GCode()
for laserin_power in [75, 100, 125, 150, 175, 200, 255]:
    print(f"Power Test: {laserin_power}")
    for x_spacing in [0, 0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.5]:
        for Ys, laserin_speed in zip([[10,20],[20,30],[30,40], [40,50], [50,60]], [50, 100, 150, 200, 250]):
            X_pos+=x_spacing
            X_pos=np.round(X_pos, 4)
            #
            doe.G0(X=X_pos, Y=Ys[0])
            #
            doe.M3(S=int(laserin_power))
            doe.G1(X=X_pos, Y=Ys[0], F=laserin_speed)
            doe.G1(X=X_pos,Y=Ys[1], F=laserin_speed)
            doe.M5()
            #
            doe.G0(X=X_pos, Y=Ys[1])
            #
            print(f"\tX:{X_pos}\tY:{Ys[0]}:{Ys[1]}\tFeed: {laserin_speed}")
    X_pos+=5
    

Power Test: 75
	X:10	Y:10:20	Feed: 50
	X:10	Y:20:30	Feed: 100
	X:10	Y:30:40	Feed: 150
	X:10	Y:40:50	Feed: 200
	X:10	Y:50:60	Feed: 250
	X:10.1	Y:10:20	Feed: 50
	X:10.2	Y:20:30	Feed: 100
	X:10.3	Y:30:40	Feed: 150
	X:10.4	Y:40:50	Feed: 200
	X:10.5	Y:50:60	Feed: 250
	X:10.75	Y:10:20	Feed: 50
	X:11.0	Y:20:30	Feed: 100
	X:11.25	Y:30:40	Feed: 150
	X:11.5	Y:40:50	Feed: 200
	X:11.75	Y:50:60	Feed: 250
	X:12.25	Y:10:20	Feed: 50
	X:12.75	Y:20:30	Feed: 100
	X:13.25	Y:30:40	Feed: 150
	X:13.75	Y:40:50	Feed: 200
	X:14.25	Y:50:60	Feed: 250
	X:15.0	Y:10:20	Feed: 50
	X:15.75	Y:20:30	Feed: 100
	X:16.5	Y:30:40	Feed: 150
	X:17.25	Y:40:50	Feed: 200
	X:18.0	Y:50:60	Feed: 250
	X:19.0	Y:10:20	Feed: 50
	X:20.0	Y:20:30	Feed: 100
	X:21.0	Y:30:40	Feed: 150
	X:22.0	Y:40:50	Feed: 200
	X:23.0	Y:50:60	Feed: 250
	X:24.25	Y:10:20	Feed: 50
	X:25.5	Y:20:30	Feed: 100
	X:26.75	Y:30:40	Feed: 150
	X:28.0	Y:40:50	Feed: 200
	X:29.25	Y:50:60	Feed: 250
	X:30.75	Y:10:20	Feed: 50
	X:32.25	Y:20:30	Feed: 100
	X:33.75	Y:30:40	Feed: 150

In [11]:
import grbl

In [ ]:
with grbl.Grbl(port="/dev/ttyUSB0") as cnc:
    cnc.reset()
    cnc.home()
    cnc.run(init())
    cnc.run(doe)
    cnc.home()

 62%|██████▏   | 1433/2310 [31:33<06:11,  2.36it/s]  

^C


In [ ]:
doe